In [1]:
import numpy as np
from keras.layers import Dense, Activation, LeakyReLU, BatchNormalization
from keras.models import Sequential
from keras.losses import MeanSquaredError, BinaryCrossentropy
from keras.regularizers import l1
from keras.datasets import boston_housing
from keras.utils import normalize

Using TensorFlow backend.


In [2]:
def standardize(X, eps=1e-10):
    m = X.mean(axis=0, keepdims=True)
    s = X.std(axis=0, keepdims=True)
    s[np.where(s == 0)] = eps
    return (X - m) / s

In [3]:
(X_train, y_train), (X_test, y_test) = boston_housing.load_data()
X_train = normalize(X_train, axis=0)
X_test = normalize(X_test, axis=0)
y_train = y_train[:, np.newaxis]
y_test = y_test[:, np.newaxis]

d = X_train.shape[1]
# enc_d = 32
enc_d = d

In [4]:
model = Sequential()
model.add(Dense(8, input_dim=d, activation="relu"))
model.add(BatchNormalization())
model.add(Dense(4, activation="relu"))
model.add(Dense(1))
model.compile(optimizer="adam", loss=MeanSquaredError(), metrics=["accuracy"])
history = model.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_test, y_test))

Train on 404 samples, validate on 102 samples
Epoch 1/50
404/404 [==============================] - 1s 1ms/step - loss: 602.4036 - accuracy: 0.0000e+00 - val_loss: 617.4592 - val_accuracy: 0.0000e+00
Epoch 2/50
404/404 [==============================] - 0s 116us/step - loss: 597.2856 - accuracy: 0.0000e+00 - val_loss: 615.7508 - val_accuracy: 0.0000e+00
Epoch 3/50
404/404 [==============================] - 0s 119us/step - loss: 594.0826 - accuracy: 0.0000e+00 - val_loss: 614.4809 - val_accuracy: 0.0000e+00
Epoch 4/50
404/404 [==============================] - 0s 124us/step - loss: 591.4284 - accuracy: 0.0000e+00 - val_loss: 613.4358 - val_accuracy: 0.0000e+00
Epoch 5/50
404/404 [==============================] - 0s 119us/step - loss: 589.0320 - accuracy: 0.0000e+00 - val_loss: 612.3980 - val_accuracy: 0.0000e+00
Epoch 6/50
404/404 [==============================] - 0s 116us/step - loss: 586.6622 - accuracy: 0.0000e+00 - val_loss: 611.3305 - val_accuracy: 0.0000e+00
Epoch 7/50
404/404 [

In [5]:
encoder = Sequential()
encoder.add(Dense(enc_d, input_dim=d, activation="relu", activity_regularizer=l1(10e-5)))

decoder = Sequential()
decoder.add(Dense(d, activation="sigmoid", input_dim=enc_d))

auto_enc = Sequential()
auto_enc.add(encoder)
auto_enc.add(decoder)
auto_enc.compile(optimizer="adadelta", loss=BinaryCrossentropy(), metrics=["accuracy"])
hist = auto_enc.fit(X_train, X_train, batch_size=32, epochs=50, validation_data=(X_test, X_test))

Train on 404 samples, validate on 102 samples
Epoch 1/50
404/404 [==============================] - 0s 352us/step - loss: 0.6897 - accuracy: 0.0550 - val_loss: 0.6851 - val_accuracy: 0.0490
Epoch 2/50
404/404 [==============================] - 0s 89us/step - loss: 0.6787 - accuracy: 0.1015 - val_loss: 0.6733 - val_accuracy: 0.0777
Epoch 3/50
404/404 [==============================] - 0s 89us/step - loss: 0.6641 - accuracy: 0.1161 - val_loss: 0.6548 - val_accuracy: 0.0897
Epoch 4/50
404/404 [==============================] - 0s 92us/step - loss: 0.6370 - accuracy: 0.1150 - val_loss: 0.6157 - val_accuracy: 0.1176
Epoch 5/50
404/404 [==============================] - 0s 87us/step - loss: 0.5774 - accuracy: 0.1266 - val_loss: 0.5324 - val_accuracy: 0.1237
Epoch 6/50
404/404 [==============================] - 0s 84us/step - loss: 0.4630 - accuracy: 0.1293 - val_loss: 0.4087 - val_accuracy: 0.1237
Epoch 7/50
404/404 [==============================] - 0s 84us/step - loss: 0.3247 - accuracy: 0

In [6]:
X_est = encoder.predict(X_test)

In [7]:
X_est[0]

array([0.6727468 , 0.27171284, 0.        , 0.        , 0.        ,
       0.        , 0.        , 2.3693194 , 1.249168  , 0.        ,
       0.11553927, 1.2314316 , 0.08897564], dtype=float32)